In [1]:
import requests
import json
import time
import copy
import re
import math
import unicodedata
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import time

In [2]:
def web_request(method_name, url, dict_data, is_urlencoded=True):
    """Web GET or POST request를 호출 후 그 결과를 dict형으로 반환 """
    method_name = method_name.upper()  # 메소드이름을 대문자로 바꾼다
    if method_name not in ('GET', 'POST'):
        raise Exception('method_name is GET or POST plz...')

    if method_name == 'GET':  # GET방식인 경우
        response = requests.get(url=url, params=dict_data,headers={'Content-Type': 'application/x-www-form-urlencoded'
                                              , 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.26'
                                              })
    elif method_name == 'POST':  # POST방식인 경우
        if is_urlencoded is True:
            response = requests.post(url=url, data=dict_data,
                                     headers={'Content-Type': 'application/x-www-form-urlencoded'
                                              , 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.26'
                                              }
                                    )
        else:
            response = requests.post(url=url, data=json.dumps(dict_data), headers={'Content-Type': 'application/json'})

    dict_meta = {'status_code': response.status_code, 'ok': response.ok, 'encoding': response.encoding,
                 'Content-Type': response.headers['Content-Type']}
    if 'json' in str(response.headers['Content-Type']):  # JSON 형태인 경우
        return {**dict_meta, **response.json()}
    else:  # 문자열 형태인 경우
        return {**dict_meta, **{'text': response.text}}

In [106]:
def getReserveMainInfoByHtml(list_html):
    reserve_info_list = []
    for item in list_html.select('div.tableA tr'):
            #서비스명	장소명	접수기간/이용기간	이용요금	접수상태	신청방법	예약방법
            #print(inside_item.select('div.sch-rslt'))
            if len(item.select('td')) > 0:
                reserve_info = {}
                reserve_id = item.select_one('a')['onclick']
                reserve_id = reserve_id[reserve_id.index("('")+2:reserve_id.rindex("')")]
                reserve_info['예약ID'] = reserve_id
                # 서비스명, 장소명
                reserve_info['서비스명'] = item.select('div.sch-rslt')[0].text
                reserve_info['장소명'] = item.select('div.sch-rslt')[1].text
                ## 접수기간/이용기간
                reserveTIme = item.select_one('td.tbl_date').text
                reserve_info['접수기간'] = reserveTIme[reserveTIme.find(' ', 20, 30):].strip()
                reserve_info['이용기간'] = reserveTIme[:reserveTIme.find(' ', 20, 30)].strip()
                ## 이용요금, 접수상태
                reserve_info['이용요금'] = item.select('td.tbl_label span.bd_label')[0].text.strip()
                reserve_info['접수상태'] = item.select('td.tbl_label span.bd_label')[1].text.strip()
                ##신청방법(오류날 확률이 젤 높음)
                reserve_info['신청방법'] = item.select('td')[5].text.strip()
                ##예약방법
                reserve_info['예약방법'] = item.select_one('span.bd_ico').text.strip()
                if '외부사이트' in reserve_info['예약방법']:
                    reserve_info['외부링크여부'] = 'Y'
                    reserve_info['외부링크'] = reserve_id
                else:
                    reserve_info['외부링크여부'] = 'N'
                    reserve_info['외부링크'] = None
                reserve_info_list.append(reserve_info)
    return reserve_info_list

In [ ]:
def getReserveInfoWithOutKeyName(): ## for backup
    for item in list_html.select('div.tableA tr'):
        if len(item.select('td')) > 0:
            #print('#서비스명	장소명	접수기간/이용기간	이용요금	접수상태	신청방법	예약방법')
            start_num = 0
            for inside_item in item.select('td'):
                if start_num == 0 :
                    reserve_id = inside_item.select_one('a')['onclick']
                    reserve_id = reserve_id[reserve_id.index("('")+2:reserve_id.rindex("')")]
                else :
                    print(inside_item.text.strip().replace('\n',','))
                start_num +=1

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [109]:
LIST_PAGE = 'https://yeyak.seoul.go.kr/web/search/selectPageListDetailSearch.do'
DETAIL_PAGE = 'https://yeyak.seoul.go.kr/web/reservation/selectReservView.do'
RESERVE_CAL = 'https://yeyak.seoul.go.kr/web/reservation/selectListReservCalAjax.do'
REQ_GET = 'GET'
REQ_POST = 'POST'
list_param = {'code':'T200','currentPage': '1'}
detail_param = {'rsv_svc_id':'S230210163043376068','code':'T500'}

In [7]:
code=T100 # 체육시설
T500 ## 공간시설
T200 ## 문화체험
T000 ## 교육강좌
T400 ## 진료복지
#function fnGoPage(currentPage) {
#		$("#currentPage").val(currentPage);
#		$("#sch_order").val($("#sel_order").val());
#		$("#search2").attr({action : "/web/search/selectPageListDetailSearchImg.do?currentPage="+currentPage ,method : 'post'}).submit();
#	}

In [110]:
detail_res = web_request(REQ_POST, DETAIL_PAGE, {'sysToday': '20230211', 'rsv_svc_id': 'S230210163043376068'})
detail_html = bs(detail_res['text'], "html.parser")

In [ ]:
#rate_box
# 예약시간은 cors 정책을 우회해야할듯

In [ ]:
cal_res = web_request(REQ_POST, DETAIL_PAGE, detail_param)
#cal_html = bs(cal_res['text'], "html.parser")

In [211]:
detail_html.select_one('div.img_box')

<div class="img_box">
<img alt="약수동 주민커뮤니티센터(약수터) 강의실2" onerror="this.src='/common/img/ko/no_img.png'" src="/cmsdata/web_upload/svc/20230210/1676014242997GCLTWF0TM4BGEYDQLQP9OYHK3.jpg"/>
<button class="btn_img pop_img" data-popopen="pop_img" type="button"><span class="hide">이미지 크게 보기</span></button>
<div class="rate_box" style="cursor: pointer;">
<span class="hide">별점0점</span>
<span class="star">
							별점1점
							</span>
<span class="star">
							별점2점
							</span>
<span class="star">
							별점3점
							</span>
<span class="star">
							별점4점
							</span>
<span class="star">
							별점5점
							</span>
<b class="txt">(<span class="fc_orange">0</span> 건)</b>
</div>
</div>

In [208]:
detail_html.select('div.rate_box')

[<div class="rate_box" style="cursor: pointer;">
 <span class="hide">별점0점</span>
 <span class="star">
 							별점1점
 							</span>
 <span class="star">
 							별점2점
 							</span>
 <span class="star">
 							별점3점
 							</span>
 <span class="star">
 							별점4점
 							</span>
 <span class="star">
 							별점5점
 							</span>
 <b class="txt">(<span class="fc_orange">0</span> 건)</b>
 </div>,
 <div class="rate_box">
 <button class="star" id="1" title="" type="button">별점1점</button>
 <button class="star" id="2" title="" type="button">별점2점</button>
 <button class="star" id="3" title="" type="button">별점3점</button>
 <button class="star" id="4" title="" type="button">별점4점</button>
 <button class="star" id="5" title="" type="button">별점5점</button>
 <span class="hide"></span>
 </div>]

In [ ]:
rate_box

In [206]:
center_info = {}
for item in detail_html.select('div.con_box ul.dt_top_list li'):
    #print(item.attrs)
    key = item.select('b')[0].text
    value= item.prettify()
    value = cleanhtml(value[value.index('</b>')+4:])
    value = value.strip().replace('\t', '').replace('\n', '').replace('\r', ' ')
    center_info[key] = value
for tag in detail_html.find_all('input', type='hidden', attrs={'name': 'x'}):
    center_info['경도'] = float(tag['value'])

for tag in detail_html.find_all('input', type='hidden', attrs={'name': 'y'}):
    center_info['위도'] = float(tag['value'])
#for tag in detail_html.find_all('input', type='hidden', attrs={'name': 'mgis_cdata'}):
print(center_info)

{'대상': '제한없음', '장소': '약수동 주민센터  (면적 : 13.6)   지도보기', '이용기간': '2023.02.10 ~ 2023.12.31', '접수기간': '2023.02.10 09:00 ~ 2023.12.31 18:00  이용일로부터 30 일 전 9시 ~     이용예정 1 일 전     18시   까지   (예: 03.14일 예약신청은 30일전(02.12) 9시부터 개시)', '선별방법': '심사', '모집정원': '1 팀', '신청제한': '1회에 1팀 까지 신청가능', '취소기간': '이용일     1일 전까지', '이용요금': '무료', '예약방법': '전화,현장/방문', '문의전화': '약수동주민센터 /   02-3396-8522', '경도': 127.00899466106634, '위도': 37.552472907891136}


In [ ]:
디테일
<div class="tableA">
<table class="tbl_mob">

총건수
<div>
<h3 class="title_dep1">총 <span class="text_red"> 417 </span>건
		<div class="right_box">
    
    # 한페이지에 목록은 6개